In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
from matplotlib.patches import Rectangle
import matplotlib.patches as mpatches

# Set consistent style for all plots
plt.style.use('default')
sns.set_palette("husl")

# Global plot settings for thesis quality
PLOT_CONFIG = {
    'figure.dpi': 300,
    'savefig.dpi': 300,
    'font.size': 16,
    'axes.titlesize': 20,
    'axes.labelsize': 18,
    'xtick.labelsize': 16,
    'ytick.labelsize': 16,
    'legend.fontsize': 16,
    'figure.figsize': (12, 8),
    'lines.linewidth': 3,
    'lines.markersize': 10,
    'grid.alpha': 0.3
}

# Apply settings
plt.rcParams.update(PLOT_CONFIG)

# Consistent color scheme
COLORS = {
    'BNScale': '#2E86AB',      # Blue
    'MagnitudeL2': '#A23B72',  # Purple/Pink
    'Random': '#F18F01',       # Orange
    'MobileNetV2': '#2E86AB',
    'ResNet-18': '#A23B72',
    'MLP': '#F18F01',
    'LSTM': '#C73E1D'
}

def load_all_results():
    """Load all experimental results from JSON files"""
    try:
        with open('mobileNetV2_results.json', 'r') as f:
            mobilenet_results = json.load(f)
        with open('resnet18_results.json', 'r') as f:
            resnet_results = json.load(f)
        with open('mlp_results.json', 'r') as f:
            mlp_results = json.load(f)
        with open('lstm_results.json', 'r') as f:
            lstm_results = json.load(f)
        return mobilenet_results, resnet_results, mlp_results, lstm_results
    except FileNotFoundError as e:
        print(f"Error loading files: {e}")
        return None, None, None, None

def results_to_dataframe(results, model_name, task_type='classification'):
    """Convert results dictionary to DataFrame"""
    data = []
    for strategy, strategy_results in results.items():
        for sparsity_str, metrics in strategy_results.items():
            sparsity = float(sparsity_str)
            row = {
                'model': model_name,
                'strategy': strategy,
                'sparsity': sparsity,
                'sparsity_percent': sparsity * 100,
                'macs': float(metrics['macs']),
                'macs_millions': float(metrics['macs']) / 1e6,
                'params': int(metrics['params']),
                'params_millions': int(metrics['params']) / 1e6,
                'size_mb': float(metrics['size_mb']),
                'loss': float(metrics['loss'])
            }

            if task_type == 'classification':
                row['accuracy'] = float(metrics['accuracy'])
            else:  # regression
                row['mse'] = float(metrics.get('mse', metrics.get('loss', 0)))
                row['mae'] = float(metrics.get('mae', 0))

            data.append(row)

    return pd.DataFrame(data)

def save_plot(filename, output_dir='thesis_plots'):
    """Save plot with consistent settings"""
    os.makedirs(output_dir, exist_ok=True)
    filepath = os.path.join(output_dir, filename)
    plt.tight_layout()
    plt.savefig(filepath, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close()
    print(f"✅ Saved: {filepath}")

# 1. Baseline Performance Comparison
def plot_baseline_comparison(mobilenet_df, resnet_df, mlp_df, lstm_df):
    """Plot baseline performance across all models"""
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

    # Get baseline data
    baselines = []
    for df, model in [(mobilenet_df, 'MobileNetV2'), (resnet_df, 'ResNet-18'),
                      (mlp_df, 'MLP'), (lstm_df, 'LSTM')]:
        baseline = df[df['sparsity'] == 0.0].iloc[0]
        baselines.append({
            'model': model,
            'accuracy': baseline.get('accuracy', None),
            'mse': baseline.get('mse', None),
            'macs_millions': baseline['macs_millions'],
            'size_mb': baseline['size_mb']
        })

    baseline_df = pd.DataFrame(baselines)

    # Accuracy comparison (CNN models)
    cnn_models = baseline_df[baseline_df['accuracy'].notna()]
    ax1.bar(cnn_models['model'], cnn_models['accuracy'],
            color=[COLORS['MobileNetV2'], COLORS['ResNet-18']], alpha=0.8)
    ax1.set_ylabel('Accuracy (%)')
    ax1.set_title('Baseline Accuracy Comparison')
    ax1.grid(True, alpha=0.3)

    # MSE comparison (Time-series models)
    ts_models = baseline_df[baseline_df['mse'].notna()]
    ax2.bar(ts_models['model'], ts_models['mse'],
            color=[COLORS['MLP'], COLORS['LSTM']], alpha=0.8)
    ax2.set_ylabel('MSE')
    ax2.set_title('Baseline MSE Comparison')
    ax2.grid(True, alpha=0.3)

    # MACs comparison
    ax3.bar(baseline_df['model'], baseline_df['macs_millions'],
            color=[COLORS[model] for model in baseline_df['model']], alpha=0.8)
    ax3.set_ylabel('MACs (Millions)')
    ax3.set_title('Baseline Computational Cost')
    ax3.set_yscale('log')
    ax3.grid(True, alpha=0.3)

    # Model size comparison
    ax4.bar(baseline_df['model'], baseline_df['size_mb'],
            color=[COLORS[model] for model in baseline_df['model']], alpha=0.8)
    ax4.set_ylabel('Model Size (MB)')
    ax4.set_title('Baseline Model Size')
    ax4.set_yscale('log')
    ax4.grid(True, alpha=0.3)

    plt.suptitle('Baseline Performance Comparison Across All Models', fontsize=24, fontweight='bold')
    save_plot('01_baseline_comparison.png')

# 2. MobileNetV2 Accuracy vs Sparsity
def plot_mobilenetv2_accuracy_sparsity(mobilenet_df):
    """Plot MobileNetV2 accuracy vs sparsity"""
    plt.figure(figsize=(12, 8))

    for strategy in mobilenet_df['strategy'].unique():
        data = mobilenet_df[mobilenet_df['strategy'] == strategy].sort_values('sparsity')
        plt.plot(data['sparsity_percent'], data['accuracy'], 'o-',
                linewidth=3, markersize=10, label=strategy, color=COLORS[strategy])

    plt.xlabel('Sparsity (%)')
    plt.ylabel('Accuracy (%)')
    plt.title('MobileNetV2: Accuracy vs Sparsity Level', fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.ylim(85, 91)

    save_plot('02_mobilenetv2_accuracy_vs_sparsity.png')

# 3. ResNet-18 Accuracy vs Sparsity
def plot_resnet18_accuracy_sparsity(resnet_df):
    """Plot ResNet-18 accuracy vs sparsity"""
    plt.figure(figsize=(12, 8))

    for strategy in resnet_df['strategy'].unique():
        data = resnet_df[resnet_df['strategy'] == strategy].sort_values('sparsity')
        plt.plot(data['sparsity_percent'], data['accuracy'], 'o-',
                linewidth=3, markersize=10, label=strategy, color=COLORS[strategy])

    plt.xlabel('Sparsity (%)')
    plt.ylabel('Accuracy (%)')
    plt.title('ResNet-18: Accuracy vs Sparsity Level', fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.ylim(84, 91)

    save_plot('03_resnet18_accuracy_vs_sparsity.png')

# 4. MLP Performance vs Sparsity
def plot_mlp_performance_sparsity(mlp_df):
    """Plot MLP MSE and MAE vs sparsity"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

    # MSE plot
    for strategy in mlp_df['strategy'].unique():
        data = mlp_df[mlp_df['strategy'] == strategy].sort_values('sparsity')
        ax1.plot(data['sparsity_percent'], data['mse'], 'o-',
                linewidth=3, markersize=10, label=strategy, color=COLORS[strategy])

    ax1.set_xlabel('Sparsity (%)')
    ax1.set_ylabel('MSE')
    ax1.set_title('MLP: MSE vs Sparsity Level', fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # MAE plot
    for strategy in mlp_df['strategy'].unique():
        data = mlp_df[mlp_df['strategy'] == strategy].sort_values('sparsity')
        ax2.plot(data['sparsity_percent'], data['mae'], 'o-',
                linewidth=3, markersize=10, label=strategy, color=COLORS[strategy])

    ax2.set_xlabel('Sparsity (%)')
    ax2.set_ylabel('MAE')
    ax2.set_title('MLP: MAE vs Sparsity Level', fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    save_plot('04_mlp_performance_vs_sparsity.png')

# 5. LSTM Performance vs Sparsity
def plot_lstm_performance_sparsity(lstm_df):
    """Plot LSTM MSE and MAE vs sparsity"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

    # MSE plot
    for strategy in lstm_df['strategy'].unique():
        data = lstm_df[lstm_df['strategy'] == strategy].sort_values('sparsity')
        ax1.plot(data['sparsity_percent'], data['mse'], 'o-',
                linewidth=3, markersize=10, label=strategy, color=COLORS[strategy])

    ax1.set_xlabel('Sparsity (%)')
    ax1.set_ylabel('MSE')
    ax1.set_title('LSTM: MSE vs Sparsity Level', fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # MAE plot
    for strategy in lstm_df['strategy'].unique():
        data = lstm_df[lstm_df['strategy'] == strategy].sort_values('sparsity')
        ax2.plot(data['sparsity_percent'], data['mae'], 'o-',
                linewidth=3, markersize=10, label=strategy, color=COLORS[strategy])

    ax2.set_xlabel('Sparsity (%)')
    ax2.set_ylabel('MAE')
    ax2.set_title('LSTM: MAE vs Sparsity Level', fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    save_plot('05_lstm_performance_vs_sparsity.png')

# 6. MobileNetV2 Efficiency Frontier
def plot_mobilenetv2_efficiency_frontier(mobilenet_df):
    """Plot MobileNetV2 Pareto frontier"""
    plt.figure(figsize=(12, 8))

    for strategy in mobilenet_df['strategy'].unique():
        data = mobilenet_df[mobilenet_df['strategy'] == strategy].sort_values('macs_millions')
        plt.scatter(data['macs_millions'], data['accuracy'],
                   s=150, label=strategy, color=COLORS[strategy], alpha=0.8,
                   edgecolors='black', linewidth=1)
        plt.plot(data['macs_millions'], data['accuracy'],
                linestyle='--', alpha=0.7, linewidth=2, color=COLORS[strategy])

    plt.xlabel('MACs (Millions)')
    plt.ylabel('Accuracy (%)')
    plt.title('MobileNetV2: Accuracy vs Computational Cost (Pareto Frontier)', fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)

    save_plot('06_mobilenetv2_efficiency_frontier.png')

# 7. ResNet-18 Efficiency Frontier
def plot_resnet18_efficiency_frontier(resnet_df):
    """Plot ResNet-18 Pareto frontier"""
    plt.figure(figsize=(12, 8))

    for strategy in resnet_df['strategy'].unique():
        data = resnet_df[resnet_df['strategy'] == strategy].sort_values('macs_millions')
        plt.scatter(data['macs_millions'], data['accuracy'],
                   s=150, label=strategy, color=COLORS[strategy], alpha=0.8,
                   edgecolors='black', linewidth=1)
        plt.plot(data['macs_millions'], data['accuracy'],
                linestyle='--', alpha=0.7, linewidth=2, color=COLORS[strategy])

    plt.xlabel('MACs (Millions)')
    plt.ylabel('Accuracy (%)')
    plt.title('ResNet-18: Accuracy vs Computational Cost (Pareto Frontier)', fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)

    save_plot('07_resnet18_efficiency_frontier.png')

# 8. CNN Models Comparison
def plot_cnn_comparison(mobilenet_df, resnet_df):
    """Compare CNN models performance"""
    plt.figure(figsize=(12, 8))

    # Plot MobileNetV2 (MagnitudeL2 strategy)
    mob_data = mobilenet_df[mobilenet_df['strategy'] == 'MagnitudeL2'].sort_values('sparsity')
    plt.plot(mob_data['sparsity_percent'], mob_data['accuracy'], 'o-',
            linewidth=3, markersize=10, label='MobileNetV2', color=COLORS['MobileNetV2'])

    # Plot ResNet-18 (MagnitudeL2 strategy)
    res_data = resnet_df[resnet_df['strategy'] == 'MagnitudeL2'].sort_values('sparsity')
    plt.plot(res_data['sparsity_percent'], res_data['accuracy'], 's-',
            linewidth=3, markersize=10, label='ResNet-18', color=COLORS['ResNet-18'])

    plt.xlabel('Sparsity (%)')
    plt.ylabel('Accuracy (%)')
    plt.title('CNN Architectures Comparison: Accuracy vs Sparsity (MagnitudeL2)', fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)

    save_plot('08_cnn_comparison.png')

# 9. Time-Series Models Comparison
def plot_timeseries_comparison(mlp_df, lstm_df):
    """Compare time-series models performance"""
    plt.figure(figsize=(12, 8))

    # Plot MLP (MagnitudeL2 strategy)
    mlp_data = mlp_df[mlp_df['strategy'] == 'MagnitudeL2'].sort_values('sparsity')
    plt.plot(mlp_data['sparsity_percent'], mlp_data['mse'], 'o-',
            linewidth=3, markersize=10, label='MLP', color=COLORS['MLP'])

    # Plot LSTM (MagnitudeL2 strategy)
    lstm_data = lstm_df[lstm_df['strategy'] == 'MagnitudeL2'].sort_values('sparsity')
    plt.plot(lstm_data['sparsity_percent'], lstm_data['mse'], 's-',
            linewidth=3, markersize=10, label='LSTM', color=COLORS['LSTM'])

    plt.xlabel('Sparsity (%)')
    plt.ylabel('MSE')
    plt.title('Time-Series Models Comparison: MSE vs Sparsity (MagnitudeL2)', fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)

    save_plot('09_timeseries_comparison.png')

# 10. Model Size Reduction Analysis
def plot_model_size_reduction(mobilenet_df, resnet_df, mlp_df, lstm_df):
    """Plot model size reduction across all models"""
    plt.figure(figsize=(14, 8))

    models = ['MobileNetV2', 'ResNet-18', 'MLP', 'LSTM']
    sparsity_levels = [0.0, 0.2, 0.5, 0.7]

    x = np.arange(len(models))
    width = 0.2

    for i, sparsity in enumerate(sparsity_levels):
        sizes = []
        for df in [mobilenet_df, resnet_df, mlp_df, lstm_df]:
            # Use MagnitudeL2 strategy for comparison
            if 'MagnitudeL2' in df['strategy'].unique():
                size = df[(df['strategy'] == 'MagnitudeL2') &
                         (df['sparsity'] == sparsity)]['size_mb'].values[0]
            else:  # For models without MagnitudeL2, use first available strategy
                size = df[df['sparsity'] == sparsity]['size_mb'].values[0]
            sizes.append(size)

        bars = plt.bar(x + i*width, sizes, width,
                      label=f'{int(sparsity*100)}% Sparsity', alpha=0.8)

        # Add value labels on bars
        for bar, size in zip(bars, sizes):
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                    f'{size:.1f}', ha='center', va='bottom', fontsize=12)

    plt.xlabel('Model Architecture')
    plt.ylabel('Model Size (MB)')
    plt.title('Model Size Reduction Across Architectures', fontweight='bold')
    plt.xticks(x + width * 1.5, models)
    plt.legend()
    plt.grid(True, alpha=0.3, axis='y')
    plt.yscale('log')

    save_plot('10_model_size_reduction.png')

# 11. MACs Reduction Analysis
def plot_macs_reduction(mobilenet_df, resnet_df, mlp_df, lstm_df):
    """Plot MACs reduction across all models"""
    plt.figure(figsize=(12, 8))

    models_data = [
        (mobilenet_df, 'MobileNetV2', COLORS['MobileNetV2']),
        (resnet_df, 'ResNet-18', COLORS['ResNet-18']),
        (mlp_df, 'MLP', COLORS['MLP']),
        (lstm_df, 'LSTM', COLORS['LSTM'])
    ]

    for df, model_name, color in models_data:
        # Use MagnitudeL2 strategy if available
        if 'MagnitudeL2' in df['strategy'].unique():
            data = df[df['strategy'] == 'MagnitudeL2'].sort_values('sparsity')
        else:
            data = df[df['strategy'] == df['strategy'].unique()[0]].sort_values('sparsity')

        baseline_macs = data[data['sparsity'] == 0.0]['macs_millions'].values[0]
        macs_retained = (data['macs_millions'] / baseline_macs) * 100

        plt.plot(data['sparsity_percent'], macs_retained, 'o-',
                linewidth=3, markersize=10, label=model_name, color=color)

    plt.xlabel('Sparsity (%)')
    plt.ylabel('MACs Retained (%)')
    plt.title('Computational Cost Reduction Across Models', fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.ylim(0, 105)

    # Add reference lines
    plt.axhline(y=50, color='gray', linestyle=':', alpha=0.7, linewidth=2)
    plt.text(35, 52, '50% MACs Retained', fontsize=14, color='gray')

    save_plot('11_macs_reduction.png')

# 12. Performance Retention Analysis
def plot_performance_retention(mobilenet_df, resnet_df, mlp_df, lstm_df):
    """Plot performance retention across models"""
    plt.figure(figsize=(14, 8))

    sparsity_levels = [0.2, 0.5, 0.7]
    models = ['MobileNetV2', 'ResNet-18', 'MLP', 'LSTM']

    x = np.arange(len(sparsity_levels))
    width = 0.2

    for i, (df, model_name, metric) in enumerate([
        (mobilenet_df, 'MobileNetV2', 'accuracy'),
        (resnet_df, 'ResNet-18', 'accuracy'),
        (mlp_df, 'MLP', 'mse'),
        (lstm_df, 'LSTM', 'mse')
    ]):
        retention_rates = []

        # Use MagnitudeL2 if available
        strategy = 'MagnitudeL2' if 'MagnitudeL2' in df['strategy'].unique() else df['strategy'].unique()[0]
        baseline = df[(df['strategy'] == strategy) & (df['sparsity'] == 0.0)].iloc[0]

        for sparsity in sparsity_levels:
            pruned = df[(df['strategy'] == strategy) & (df['sparsity'] == sparsity)].iloc[0]

            if metric == 'accuracy':
                retention = (pruned['accuracy'] / baseline['accuracy']) * 100
            else:  # For MSE, inverse retention (lower is better)
                retention = (baseline['mse'] / pruned['mse']) * 100

            retention_rates.append(retention)

        plt.bar(x + i*width, retention_rates, width,
               label=model_name, alpha=0.8, color=COLORS[model_name])

    plt.xlabel('Sparsity Level (%)')
    plt.ylabel('Performance Retention (%)')
    plt.title('Performance Retention Across Models and Sparsity Levels', fontweight='bold')
    plt.xticks(x + width * 1.5, [f'{int(s*100)}%' for s in sparsity_levels])
    plt.legend()
    plt.grid(True, alpha=0.3, axis='y')

    # Add reference line at 100%
    plt.axhline(y=100, color='black', linestyle='--', alpha=0.5, linewidth=2)

    save_plot('12_performance_retention.png')

# 13. Strategy Effectiveness Heatmap
def plot_strategy_effectiveness_heatmap(mobilenet_df, resnet_df):
    """Plot strategy effectiveness heatmap for CNN models"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

    # MobileNetV2 heatmap
    pivot_mob = mobilenet_df.pivot(index='strategy', columns='sparsity_percent', values='accuracy')
    baseline_mob = pivot_mob[0.0]
    normalized_mob = (pivot_mob.T / baseline_mob * 100).T

    sns.heatmap(normalized_mob, annot=True, fmt='.1f', cmap='RdYlGn',
                center=100, ax=ax1, cbar_kws={'label': 'Relative Accuracy (%)'})
    ax1.set_title('MobileNetV2: Strategy Effectiveness', fontweight='bold')
    ax1.set_xlabel('Sparsity (%)')
    ax1.set_ylabel('Pruning Strategy')

    # ResNet-18 heatmap
    pivot_res = resnet_df.pivot(index='strategy', columns='sparsity_percent', values='accuracy')
    baseline_res = pivot_res[0.0]
    normalized_res = (pivot_res.T / baseline_res * 100).T

    sns.heatmap(normalized_res, annot=True, fmt='.1f', cmap='RdYlGn',
                center=100, ax=ax2, cbar_kws={'label': 'Relative Accuracy (%)'})
    ax2.set_title('ResNet-18: Strategy Effectiveness', fontweight='bold')
    ax2.set_xlabel('Sparsity (%)')
    ax2.set_ylabel('Pruning Strategy')

    save_plot('13_strategy_effectiveness_heatmap.png')

# 14. LSTM Special Analysis
def plot_lstm_special_analysis(lstm_df):
    """Special analysis for LSTM showing minimal MACs reduction"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

    # MACs vs Sparsity
    for strategy in lstm_df['strategy'].unique():
        data = lstm_df[lstm_df['strategy'] == strategy].sort_values('sparsity')
        ax1.plot(data['sparsity_percent'], data['macs_millions'], 'o-',
                linewidth=3, markersize=10, label=strategy, color=COLORS[strategy])

    ax1.set_xlabel('Sparsity (%)')
    ax1.set_ylabel('MACs (Millions)')
    ax1.set_title('LSTM: MACs vs Sparsity (Minimal Reduction)', fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Parameters vs MSE
    for strategy in lstm_df['strategy'].unique():
        data = lstm_df[lstm_df['strategy'] == strategy].sort_values('params')
        ax2.scatter(data['params'] / 1000, data['mse'], s=150,
                   label=strategy, alpha=0.8, color=COLORS[strategy])
        ax2.plot(data['params'] / 1000, data['mse'], '--',
                alpha=0.7, linewidth=2, color=COLORS[strategy])

    ax2.set_xlabel('Parameters (Thousands)')
    ax2.set_ylabel('MSE')
    ax2.set_title('LSTM: Parameter Reduction vs Performance', fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    save_plot('14_lstm_special_analysis.png')

# 15. Edge Deployment Feasibility Analysis
def plot_edge_deployment_analysis(mobilenet_df, resnet_df, mlp_df, lstm_df):
    """Plot edge deployment feasibility based on resource constraints"""
    plt.figure(figsize=(12, 8))

    # Define edge device constraints (example thresholds)
    EDGE_CONSTRAINTS = {
        'max_macs': 10,  # 10M MACs
        'max_size': 5,   # 5MB
    }

    models_data = [
        (mobilenet_df, 'MobileNetV2'),
        (resnet_df, 'ResNet-18'),
        (mlp_df, 'MLP'),
        (lstm_df, 'LSTM')
    ]

    feasible_points = []

    for df, model_name in models_data:
        # Use MagnitudeL2 if available
        strategy = 'MagnitudeL2' if 'MagnitudeL2' in df['strategy'].unique() else df['strategy'].unique()[0]
        data = df[df['strategy'] == strategy]

        for _, row in data.iterrows():
            feasible = (row['macs_millions'] <= EDGE_CONSTRAINTS['max_macs'] and
                       row['size_mb'] <= EDGE_CONSTRAINTS['max_size'])

            color = COLORS[model_name] if feasible else 'lightgray'
            alpha = 1.0 if feasible else 0.3
            marker = 'o' if feasible else 'x'

            plt.scatter(row['macs_millions'], row['size_mb'],
                       s=150, color=color, alpha=alpha, marker=marker,
                       label=model_name if row['sparsity'] == 0.0 else "")

    # Add constraint lines
    plt.axvline(x=EDGE_CONSTRAINTS['max_macs'], color='red', linestyle='--',
                linewidth=2, alpha=0.7, label='MACs Constraint')
    plt.axhline(y=EDGE_CONSTRAINTS['max_size'], color='red', linestyle='--',
                linewidth=2, alpha=0.7, label='Size Constraint')

    plt.xlabel('MACs (Millions)')
    plt.ylabel('Model Size (MB)')
    plt.title('Edge Deployment Feasibility Analysis', fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.yscale('log')
    plt.xscale('log')

    save_plot('15_edge_deployment_analysis.png')

def main():
    """Generate all plots for Chapter 5"""
    print("🚀 Generating comprehensive thesis plots for Chapter 5...")

    # Load data
    mobilenet_results, resnet_results, mlp_results, lstm_results = load_all_results()

    if any(x is None for x in [mobilenet_results, resnet_results, mlp_results, lstm_results]):
        print("❌ Error loading data files. Please check file names and paths.")
        return

    # Convert to DataFrames
    mobilenet_df = results_to_dataframe(mobilenet_results, 'MobileNetV2', 'classification')
    resnet_df = results_to_dataframe(resnet_results, 'ResNet-18', 'classification')
    mlp_df = results_to_dataframe(mlp_results, 'MLP', 'regression')
    lstm_df = results_to_dataframe(lstm_results, 'LSTM', 'regression')

    print(f"📊 Data loaded successfully:")
    print(f"   • MobileNetV2: {len(mobilenet_df)} data points")
    print(f"   • ResNet-18: {len(resnet_df)} data points")
    print(f"   • MLP: {len(mlp_df)} data points")
    print(f"   • LSTM: {len(lstm_df)} data points")

    # Generate all plots
    plot_functions = [
        (plot_baseline_comparison, "Baseline comparison"),
        (plot_mobilenetv2_accuracy_sparsity, "MobileNetV2 accuracy vs sparsity"),
        (plot_resnet18_accuracy_sparsity, "ResNet-18 accuracy vs sparsity"),
        (plot_mlp_performance_sparsity, "MLP performance vs sparsity"),
        (plot_lstm_performance_sparsity, "LSTM performance vs sparsity"),
        (plot_mobilenetv2_efficiency_frontier, "MobileNetV2 efficiency frontier"),
        (plot_resnet18_efficiency_frontier, "ResNet-18 efficiency frontier"),
        (plot_cnn_comparison, "CNN models comparison"),
        (plot_timeseries_comparison, "Time-series models comparison"),
        (plot_model_size_reduction, "Model size reduction analysis"),
        (plot_macs_reduction, "MACs reduction analysis"),
        (plot_performance_retention, "Performance retention analysis"),
        (plot_strategy_effectiveness_heatmap, "Strategy effectiveness heatmap"),
        (plot_lstm_special_analysis, "LSTM special analysis"),
        (plot_edge_deployment_analysis, "Edge deployment feasibility")
    ]

    for i, (plot_func, description) in enumerate(plot_functions, 1):
        print(f"📈 Generating plot {i:2d}/15: {description}")
        try:
            if plot_func.__name__ in ['plot_baseline_comparison', 'plot_model_size_reduction',
                                    'plot_macs_reduction', 'plot_performance_retention',
                                    'plot_edge_deployment_analysis']:
                plot_func(mobilenet_df, resnet_df, mlp_df, lstm_df)
            elif plot_func.__name__ == 'plot_cnn_comparison':
                plot_func(mobilenet_df, resnet_df)
            elif plot_func.__name__ == 'plot_timeseries_comparison':
                plot_func(mlp_df, lstm_df)
            elif plot_func.__name__ == 'plot_strategy_effectiveness_heatmap':
                plot_func(mobilenet_df, resnet_df)
            elif 'mobilenetv2' in plot_func.__name__:
                plot_func(mobilenet_df)
            elif 'resnet18' in plot_func.__name__:
                plot_func(resnet_df)
            elif 'mlp' in plot_func.__name__:
                plot_func(mlp_df)
            elif 'lstm' in plot_func.__name__:
                plot_func(lstm_df)
        except Exception as e:
            print(f"❌ Error generating {description}: {e}")

    print(f"\n🎉 All plots generated successfully!")
    print(f"📁 Plots saved in: thesis_plots/")
    print(f"📋 Total plots: 15")
    print(f"🔧 Plot specifications:")
    print(f"   • Resolution: 300 DPI")
    print(f"   • Format: PNG")
    print(f"   • Font sizes: Title(20), Labels(18), Ticks(16)")
    print(f"   • Consistent color scheme applied")

if __name__ == "__main__":
    main()